In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
import warnings

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/cyber-attack-anomaly-scores-regression/sample_submission.csv
/kaggle/input/cyber-attack-anomaly-scores-regression/cyberattack-regression-baseline.ipynb
/kaggle/input/cyber-attack-anomaly-scores-regression/train_data.csv
/kaggle/input/cyber-attack-anomaly-scores-regression/test_data.csv
/kaggle/input/cyber-attack-anomaly-scores-regression/train_label.csv


# Load Dataset

In [3]:
train_data_df = pd.read_csv("/kaggle/input/cyber-attack-anomaly-scores-regression/train_data.csv")
train_data_df.head()

,Source IP Address,Destination IP Address,Source Port,Destination Port,Protocol,Packet Type,Traffic Type,Payload Data,Malware Indicators,Alerts/Warnings,Attack Signature,Action Taken,Severity Level,User Information,Device Information,Network Segment,Geo-location Data,Log Source,Packet Length,Id
0,54.67.170.151,95.6.233.231,1547,44666,TCP,Control,HTTP,Blanditiis impedit mollitia officia fugiat deb...,None Detected,Alert Triggered,Known Pattern A,Logged,Low,Baiju Sidhu,Mozilla/5.0 (compatible; MSIE 5.0; Windows NT ...,Segment C,"Hindupur, Tripura",Server,1421,0
1,104.13.117.72,100.174.73.224,23951,5640,ICMP,Data,DNS,Debitis quaerat sunt a. Autem repudiandae cupi...,None Detected,No Alert,Known Pattern B,Logged,Medium,Ivan Sama,Mozilla/5.0 (Windows; U; Windows NT 5.2) Apple...,Segment C,"Hazaribagh, Meghalaya",Server,1128,1
2,144.114.97.211,25.124.53.187,52284,61791,UDP,Control,FTP,Accusamus magnam amet nobis minima. Consequunt...,IoC Detected,Alert Triggered,Known Pattern A,Ignored,High,Hazel Shankar,Opera/8.73.(X11; Linux i686; ak-GH) Presto/2.9...,Segment B,"Guwahati, Andhra Pradesh",Firewall,1293,2
3,181.205.198.7,202.35.6.147,18299,52346,UDP,Data,FTP,Cupiditate dolores facilis explicabo. Vel quid...,IoC Detected,No Alert,Known Pattern B,Ignored,Low,Kiaan Dara,Mozilla/5.0 (compatible; MSIE 7.0; Windows NT ...,Segment B,"Bally, Punjab",Firewall,188,3
4,1.69.139.97,183.66.243.79,42605,16099,TCP,Control,FTP,Doloremque fugit placeat non. Voluptates rerum...,IoC Detected,No Alert,Known Pattern A,Ignored,Medium,Kartik Sen,Mozilla/5.0 (Macintosh; U; Intel Mac OS X 10_7...,Segment A,"Mira-Bhayandar, Maharashtra",Firewall,653,4


In [4]:
train_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32000 entries, 0 to 31999
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Source IP Address       32000 non-null  object
 1   Destination IP Address  32000 non-null  object
 2   Source Port             32000 non-null  int64 
 3   Destination Port        32000 non-null  int64 
 4   Protocol                32000 non-null  object
 5   Packet Type             32000 non-null  object
 6   Traffic Type            32000 non-null  object
 7   Payload Data            32000 non-null  object
 8   Malware Indicators      32000 non-null  object
 9   Alerts/Warnings         32000 non-null  object
 10  Attack Signature        32000 non-null  object
 11  Action Taken            32000 non-null  object
 12  Severity Level          32000 non-null  object
 13  User Information        32000 non-null  object
 14  Device Information      32000 non-null  object
 15  Ne

In [5]:
train_label_df = pd.read_csv("/kaggle/input/cyber-attack-anomaly-scores-regression/train_label.csv")
train_label_df.head()

,Anomaly Scores,Id
0,76.35,0
1,52.89,1
2,20.17,2
3,44.01,3
4,22.94,4


In [6]:
test_data_df = pd.read_csv("/kaggle/input/cyber-attack-anomaly-scores-regression/test_data.csv")
test_data_df.head()

,Source IP Address,Destination IP Address,Source Port,Destination Port,Protocol,Packet Type,Traffic Type,Payload Data,Malware Indicators,Alerts/Warnings,Attack Signature,Action Taken,Severity Level,User Information,Device Information,Network Segment,Geo-location Data,Log Source,Packet Length,Id
0,214.236.144.193,182.199.35.61,4999,32694,ICMP,Control,DNS,Reprehenderit est beatae distinctio sunt ex vo...,None Detected,No Alert,Known Pattern B,Ignored,Low,Amani Taneja,Mozilla/5.0 (Windows NT 5.01; da-DK; rv:1.9.1....,Segment C,"Jorhat, Chhattisgarh",Firewall,1222,0
1,218.86.56.2,43.139.186.246,55643,11859,TCP,Control,DNS,Voluptates perspiciatis magnam optio vero assu...,IoC Detected,No Alert,Known Pattern A,Ignored,Low,Parinaaz Goel,Mozilla/5.0 (Windows; U; Windows NT 6.2) Apple...,Segment B,"Yamunanagar, West Bengal",Firewall,1085,1
2,197.55.71.72,49.233.187.14,5855,4424,TCP,Control,DNS,Repellendus possimus praesentium ratione ut co...,None Detected,Alert Triggered,Known Pattern B,Logged,Medium,Ishita Jani,Mozilla/5.0 (iPad; CPU iPad OS 6_1_6 like Mac ...,Segment B,"Baranagar, Goa",Firewall,1352,2
3,63.228.148.219,49.156.152.61,65513,21972,ICMP,Data,HTTP,Voluptatibus a veritatis repudiandae veritatis...,None Detected,No Alert,Known Pattern A,Blocked,Medium,Umang Sundaram,Mozilla/5.0 (Android 3.2.1; Mobile; rv:17.0) G...,Segment B,"Allahabad, Meghalaya",Firewall,1300,3
4,62.117.101.59,69.74.24.100,36878,7510,TCP,Data,HTTP,Id velit quae. Fuga delectus itaque.\nDistinct...,None Detected,No Alert,Known Pattern A,Blocked,Medium,Taimur Ghose,Mozilla/5.0 (compatible; MSIE 7.0; Windows NT ...,Segment A,"Medininagar, Assam",Server,280,4


In [7]:
test_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Source IP Address       8000 non-null   object
 1   Destination IP Address  8000 non-null   object
 2   Source Port             8000 non-null   int64 
 3   Destination Port        8000 non-null   int64 
 4   Protocol                8000 non-null   object
 5   Packet Type             8000 non-null   object
 6   Traffic Type            8000 non-null   object
 7   Payload Data            8000 non-null   object
 8   Malware Indicators      8000 non-null   object
 9   Alerts/Warnings         8000 non-null   object
 10  Attack Signature        8000 non-null   object
 11  Action Taken            8000 non-null   object
 12  Severity Level          8000 non-null   object
 13  User Information        8000 non-null   object
 14  Device Information      8000 non-null   object
 15  Netw

In [8]:
test_label_df = pd.read_csv("/kaggle/input/cyber-attack-anomaly-scores-regression/sample_submission.csv")
test_label_df.head()

,Anomaly Scores,Id
0,0.0,0
1,0.0,1
2,0.0,2
3,0.0,3
4,0.0,4


# Modeling

In [9]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import roc_curve, auc

In [11]:
random_state = 42  # 실험결과 재현을 위해 random_state 고정

In [12]:
categorical_features = ['Source IP Address', 'Destination IP Address', 'Source Port',
                        'Destination Port', 'Protocol', 'Packet Type',
                        'Traffic Type', 'Payload Data', 'Malware Indicators', 'Alerts/Warnings',
                        'Attack Signature', 'Action Taken',
                        'Severity Level', 'User Information', 'Device Information',
                        'Network Segment', 'Geo-location Data', 'Log Source']

numerical_features = ['Packet Length']

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

X = train_data_df[categorical_features + numerical_features]
y = train_label_df['Anomaly Scores']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=random_state)

numerical_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_features),
        ('num', numerical_transformer, numerical_features)
    ])

In [13]:
X_train_processed = preprocessor.fit_transform(X_train)
X_val_processed = preprocessor.transform(X_val)

In [14]:
target_scaler = StandardScaler()
y_train = y_train.values
y_train = y_train.reshape(-1, 1)
y_val = y_val.values
y_val = y_val.reshape(-1, 1)

y_train_scaled = target_scaler.fit_transform(y_train)
y_val_scaled = target_scaler.transform(y_val)

In [15]:
lr = LinearRegression()

In [16]:
lr.fit(X_train_processed, y_train_scaled.ravel())

LinearRegression()

In [17]:
train_pred_scaled = lr.predict(X_train_processed)
train_pred = target_scaler.inverse_transform(train_pred_scaled.reshape(-1, 1))

In [18]:
val_pred_scaled = lr.predict(X_val_processed)
val_pred = target_scaler.inverse_transform(val_pred_scaled.reshape(-1, 1))

In [19]:
print(f'Train: RMSE({np.sqrt(mean_squared_error(y_train, train_pred))})')
print(f'Validation: RMSE({np.sqrt(mean_squared_error(y_val, val_pred))})')

Train: RMSE(0.005288342328736318)
Validation: RMSE(29.75454444406898)


# Test Submission

In [20]:
categorical_features = ['Source IP Address', 'Destination IP Address', 'Source Port',
                        'Destination Port', 'Protocol', 'Packet Type',
                        'Traffic Type', 'Payload Data', 'Malware Indicators', 'Alerts/Warnings',
                        'Attack Signature', 'Action Taken',
                        'Severity Level', 'User Information', 'Device Information',
                        'Network Segment', 'Geo-location Data', 'Log Source']

numerical_features = ['Packet Length']

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

X_test = test_data_df[categorical_features + numerical_features]


numerical_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_features),
        ('num', numerical_transformer, numerical_features)
    ])

In [21]:
X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

In [22]:
test_pred_scaled = lr.predict(X_test_processed)
test_pred = target_scaler.inverse_transform(test_pred_scaled.reshape(-1, 1))

In [23]:
test_label_df['Anomaly Scores'] = test_pred

In [24]:
test_label_df

,Anomaly Scores,Id
0,56.645510,0
1,44.216840,1
2,57.614689,2
3,42.918681,3
4,43.594610,4
...,...,...
7995,38.623375,7995
7996,52.475711,7996
7997,56.792206,7997
7998,42.207115,7998


In [25]:
test_label_df.to_csv('test_submission.csv', index=False)